Run using Python 3.12.4

In [215]:
import numpy as np
import pandas as pd
from numpy import savetxt
from IPython.display import display, HTML
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import random
import pickle
import os
import json

import seaborn as sns
sns.set_theme()

In [2]:
# set cell width
display(HTML("<style>.container { width:100% !important; }</style>"))

# set cell output window height
display(HTML("<style>div.output_scroll { height: 160em;} </style>"))

pd.set_option('display.width', 1000)

In [3]:
os.getcwd()

'/Users/dan_oberdorf/Documents/Northwestern/MSDS 458 Artificial Intelligence and Deep Learning/Assignment'

In [4]:
tf.__version__

'2.16.2'

### Reload Models

In [5]:
### MLP Models
model_load_list = ['mlp_base_01','mlp_base_02','mlp_base_03','mlp_static_01','mlp_static_02','mlp_static_03','mlp_dynamic_01','mlp_dynamic_02','mlp_dynamic_03']

model_wt_dict = {}

for model in model_load_list:

    file_name = f'./MLP_Models/{model}_final_wts.pkl'

    model_name = f'{model}_final_wts'

    with open(file_name,'rb') as f:
        model_wt_dict[model_name] = pickle.load(f)

In [6]:
model_wt_dict.keys()

dict_keys(['mlp_base_01_final_wts', 'mlp_base_02_final_wts', 'mlp_base_03_final_wts', 'mlp_static_01_final_wts', 'mlp_static_02_final_wts', 'mlp_static_03_final_wts', 'mlp_dynamic_01_final_wts', 'mlp_dynamic_02_final_wts', 'mlp_dynamic_03_final_wts'])

In [7]:
##### confirm that the models are loaded with both weight sets
model_wt_dict['mlp_base_01_final_wts'].keys()

dict_keys(['final_W1', 'final_W2'])

In [8]:
### Tensor Flow Models
tf_model_list = ['base_case_1','base_case_2','base_case_3','static_case_1','static_case_2','static_case_3','dyn_case_1','dyn_case_2','dyn_case_3']

tf_model_dict = {}

for model in tf_model_list:

    file_name = f'./TF_Models/{model}.keras'
    
    tf_model_dict[model] = tf.keras.models.load_model(file_name)

In [9]:
tf_model_dict['base_case_1'].summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Hidden_Layer (Dense)            │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,832 (112.63 KB)

 Trainable params: 9,610 (37.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,222 (75.09 KB)

### Load Data and Splitter

In [10]:
# Data Load and testing Test Split Functions
def load_data(data_set):

    # optionality to load two different data sets
    if data_set.lower() == 'iris':
        data = load_iris()
        X=data.data
        y=data.target
    elif data_set.lower() == 'digits':
        # data = load_digits()
        training_data = pd.read_csv('./optical+recognition+of+handwritten+digits/optdigits.tra', header=None)
        test_data = pd.read_csv('./optical+recognition+of+handwritten+digits/optdigits.tes', header=None)
        xtr = training_data.loc[:,:63]
        ytr = training_data.loc[:,64]
        xte = test_data.loc[:,:63]
        yte = test_data.loc[:,64]
        full_x_df = pd.concat([pd.DataFrame(xtr),pd.DataFrame(xte)], axis = 0)
        full_y_df = pd.concat([pd.DataFrame(ytr),pd.DataFrame(yte)], axis = 0)
        full_x_df.columns = ['x'+str(x) for x in full_x_df.columns]
        full_y_df.columns = ['Target']
        X = np.array(full_x_df)
        y = np.ndarray.flatten(np.array(full_y_df))

    print(f'Dimensions of data {X.shape}')

    # normalize data
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    # map for three output nodes
    y = pd.get_dummies(y).values
    
    return(X,y)

########################################################

def split_train_test(X, y, test_size, random_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y, shuffle=True)
    return(X_train, X_test, y_train, y_test)

In [11]:
### Seed Storage for testing
seed_split = 1

### Load Data
X, y = load_data('digits')

### Split Data
X_train, X_test, y_train, y_test = split_train_test(X, y, test_size = 0.2, random_state = seed_split)

Dimensions of data (5620, 64)


### MLP Analysis

In [12]:
class mlp_activations:

    def __init__(self, w1, w2, x_df, y_df):
        self.w1 = w1
        self.w2 = w2
        self.x_df = x_df
        self.y_df = y_df
        self.activ_df = pd.DataFrame()
        self.action = self.activations()
        

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def activations(self):
        for i, j in zip(self.x_df, self.y_df):
            Z1 = i @ self.w1
            A1 = self.sigmoid(Z1)
            A1_flat = np.ndarray.flatten(np.round(A1,decimals=3))
            Z2 = A1 @ self.w2
            A2 = self.sigmoid(Z2)
            A2_flat = np.ndarray.flatten(np.round(A2,decimals=3))
            temp_df = pd.DataFrame({'digit_y':[np.argmax(j)],'A1':[A1_flat], 'A2':[A2_flat]})
            self.activ_df = pd.concat([self.activ_df,temp_df], axis = 0, ignore_index=True)
    

In [13]:
activation_df_dict = {}

for model in model_load_list:

    model_name = f'{model}_final_wts'

    act_temp = mlp_activations(model_wt_dict[model_name]['final_W1'],
                                  model_wt_dict[model_name]['final_W2'],
                                  X_test, y_test).activ_df
    
    activation_df_dict[model] = act_temp


In [14]:
activation_df_dict['mlp_base_01']

,digit_y,A1,A2
0,6,"[0.159, 0.017, 0.005, 0.782, 0.998, 0.327, 0.5...","[0.005, 0.129, 0.001, 0.0, 0.003, 0.0, 0.0, 0...."
1,2,"[0.001, 0.101, 0.09, 0.03, 0.996, 0.944, 0.967...","[0.0, 0.077, 0.003, 0.0, 0.013, 0.001, 0.0, 0...."
2,3,"[0.0, 0.065, 0.301, 0.077, 0.777, 0.621, 0.661...","[0.002, 0.014, 0.0, 0.0, 0.0, 0.016, 0.0, 0.0,..."
3,9,"[0.001, 0.064, 0.018, 0.013, 0.999, 0.625, 0.9...","[0.066, 0.001, 0.0, 0.0, 0.001, 0.0, 0.0, 0.0,..."
4,5,"[0.007, 0.478, 0.213, 0.175, 0.988, 0.959, 0.7...","[0.006, 0.002, 0.0, 0.0, 0.465, 0.0, 0.015, 0...."
...,...,...,...
1119,8,"[0.001, 0.448, 0.002, 0.021, 0.987, 0.603, 0.9...","[0.003, 0.02, 0.0, 0.0, 0.046, 0.0, 0.001, 0.0..."
1120,7,"[0.001, 0.989, 0.12, 0.003, 0.985, 0.89, 0.939...","[0.0, 0.001, 0.001, 0.0, 0.037, 0.001, 0.0, 0...."
1121,2,"[0.0, 0.262, 0.139, 0.028, 0.999, 0.987, 0.981...","[0.003, 0.068, 0.0, 0.0, 0.003, 0.001, 0.0, 0...."
1122,8,"[0.02, 0.152, 0.141, 0.048, 0.921, 0.817, 0.82...","[0.0, 0.005, 0.001, 0.0, 0.006, 0.006, 0.0, 0...."


CMAP Options
viridis_r <br>
Reds <br>
magma <br>
crest <br>
mako <br> 
cubehelix <br>
icefire <br>
sns.diverging_palette(220, 20, as_cmap=True)

### Heatmap Maker

In [185]:
class activation_heatmap:

    def __init__(self, data_array, labels, title):
        self.data_array = data_array
        self.labels = labels
        self.title = title
        # self.action = self.graph()


    def single_graph(self):

        avg_act = np.mean(self.data_array.iloc[:,1], axis=0)
        avg_act = avg_act.reshape(avg_act.shape[0],1)
        avg_act = avg_act.T

        plt.figure(figsize = (20,2.5))

        sfig = sns.heatmap(avg_act, cmap=sns.color_palette("coolwarm", as_cmap=True), cbar = False, square = False, yticklabels=False)
        plt.title(self.title)
        plt.xticks(fontsize = 14)
        plt.tight_layout()

        return sfig.get_figure()


    def single_graph_tf(self):

        avg_act = np.mean(self.data_array, axis=0)
        avg_act = avg_act.reshape(avg_act.shape[0],1)
        avg_act = avg_act.T

        plt.figure(figsize = (20,2.5))

        sfig = sns.heatmap(avg_act, cmap=sns.color_palette("coolwarm", as_cmap=True), cbar = False, square = False, yticklabels=False)
        plt.title(self.title)
        plt.xticks(fontsize = 14)
        plt.tight_layout()

        return sfig.get_figure()


    def multi_graph(self):
        
        fig, ax = plt.subplots(len(self.data_array), sharex = True, figsize = (10,10))
        # plt.subplots_adjust(hspace = 0.01)

        for i,j in enumerate(self.data_array):

            j = j.reshape(j.shape[0],1).T
            sns.heatmap(j, ax = ax[i], cmap=sns.color_palette("coolwarm", as_cmap=True), cbar = False, square = False, yticklabels=[self.labels[i]])
            
        cbar_ax = fig.add_axes([0.92, 0.15, 0.03, 0.7])  # [left, bottom, width, height]
        fig.colorbar(ax[0].collections[0], cax=cbar_ax)

        plt.suptitle(self.title, fontsize = 18, y =0.92)

        # plt.tight_layout(h_pad=0.5, rect=(0,0,0.9,1))
        
        return fig
    

### MLP Activation Analysis

__MLP Gameplan__
1. For each model look at first 25 activations of the same number
2. Get the average activations for each model for all nine digits

In [99]:
os.getcwd()

'/Users/dan_oberdorf/Documents/Northwestern/MSDS 458 Artificial Intelligence and Deep Learning/Assignment'

In [101]:
os.chdir('./Model_Graphs')
os.getcwd()

'/Users/dan_oberdorf/Documents/Northwestern/MSDS 458 Artificial Intelligence and Deep Learning/Assignment/Model_Graphs'

In [132]:
numeric_value = 6

model_runner = 1
keys_len = len(activation_df_dict.keys())

plt.ioff()

for key, model in activation_df_dict.items():

    ## First 25 Hidden Layer
    hid_wt = model.loc[:25,'A1'].to_numpy().T
    labels = model.loc[:25,'digit_y'].values
    title = f'{key} - Activations of first 25 Characters - Hidden Layer'
    temp_fig = activation_heatmap(hid_wt, labels, title).multi_graph()
    fig_label = f'{key}_f25_hid.png'
    temp_fig.savefig(fig_label)
    plt.close()

    ## First 25 activations of a single numeric value Hidden Layer
    hid_s_wt = model.loc[model.digit_y == numeric_value,['digit_y','A1']]
    hid_s_act = hid_s_wt.A1.T.to_numpy()
    hid_s_label = hid_s_wt.digit_y.values
    title = f'{key} - Activations of first 25 Instances of "{numeric_value}" - Hidden Layer'
    temp_fig = activation_heatmap(hid_s_act[:25], hid_s_label[:25], title).multi_graph()
    fig_label = f'{key}_fs_hid.png'
    temp_fig.savefig(fig_label)
    plt.close()

    ## Mean Activations of a single numeric value Hidden Layer
    title = f'{key} - Average Activations for "{numeric_value}" - Hidden Layer'
    temp_fig = activation_heatmap(hid_s_wt, hid_s_label, title).single_graph()
    fig_label = f'{key}_fs_avg_hid.png'
    temp_fig.savefig(fig_label)
    plt.close()

    ## First 25 Output Layer
    out_wt = model.loc[:25,'A2'].to_numpy().T
    title = f'{key} - Activations of first 25 Characters - Output Layer'
    temp_fig = activation_heatmap(out_wt, labels, title).multi_graph()
    fig_label = f'{key}_f25_out.png'
    temp_fig.savefig(fig_label)
    plt.close()

    ## First 25 activations of a single numeric value Output Layer
    out_s_wt = model.loc[model.digit_y == numeric_value,['digit_y','A2']]
    out_s_act = out_s_wt.A2.T.to_numpy()
    out_s_label = out_s_wt.digit_y.values
    title = f'{key} - Activations of first 25 Instances of "{numeric_value}" - Output Layer'
    temp_fig = activation_heatmap(out_s_act[:25], out_s_label[:25], title).multi_graph()
    fig_label = f'{key}_fs_out.png'
    temp_fig.savefig(fig_label)
    plt.close()

    ## Mean Activations of a single numeric value Output Layer
    title = f'{key} - Average Activations for "{numeric_value}" - Output Layer'
    temp_fig = activation_heatmap(out_s_wt, out_s_label, title).single_graph()
    fig_label = f'{key}_fs_avg_out.png'
    temp_fig.savefig(fig_label)
    plt.close()

    # mlp_graph_dictionary[key] = temp_dict
    print(f'Finished {key} - {'{:.0%}'.format(model_runner / keys_len)} Complete.')
    model_runner += 1


Finished mlp_base_01 - 11% Complete.
Finished mlp_base_02 - 22% Complete.
Finished mlp_base_03 - 33% Complete.
Finished mlp_static_01 - 44% Complete.
Finished mlp_static_02 - 56% Complete.
Finished mlp_static_03 - 67% Complete.
Finished mlp_dynamic_01 - 78% Complete.
Finished mlp_dynamic_02 - 89% Complete.
Finished mlp_dynamic_03 - 100% Complete.


### TF

##### Testing Out How to build the activations

tf_model_dict['base_case_1'].summary()

tf.keras.backend.clear_session()

model = tf_model_dict['base_case_1']



_ = model.predict(X_test[:50])

pred = np.argmax(_,axis = 1)
pred

ans = np.argmax(y_test[:50], axis = 1)
ans

sum(pred - ans) / 50

model.get_layer('Hidden_Layer').input

model.get_layer('Hidden_Layer').output

model.get_layer('Output_Layer').input

model.get_layer('Output_Layer').output

int_out = tf.keras.Model(model.get_layer('Hidden_Layer').input, model.get_layer('Hidden_Layer').output)

int_out2 = tf.keras.Model(model.get_layer('Output_Layer').input, model.get_layer('Output_Layer').output)

int_out2(int_out(X_test[:3]))

first_item = int_out(X_test[0].reshape((1,64)))
first_item

ap = np.array(first_item)
ap = ap.flatten().reshape(128,1).T
print(ap.shape)

plt.figure(figsize = (20,1))

sns.heatmap(ap, cmap=sns.color_palette("coolwarm", as_cmap=True), cbar = False, square = False, yticklabels=False)
plt.title('TensorFlow Activations', fontsize = 18)
plt.xticks(fontsize = 14)

plt.show()

np.argmax(y_test[0])

np.argmax(y_test[1])

second_item = int_out(X_test[1].reshape((1,64)))
tp = np.array(second_item)
tp = tp.flatten().reshape(128,1).T
print(tp.shape)

plt.figure(figsize = (20,1))

sns.heatmap(tp, cmap=sns.color_palette("coolwarm", as_cmap=True), cbar = False, square = False, yticklabels=False)
plt.title('TensorFlow Activations', fontsize = 18)
plt.xticks(fontsize = 14)

plt.show()

itemx = int_out(X_test[:10].reshape((10,64)))
apa = np.array(itemx)
apa = apa.flatten().reshape(128,10).T

##### TensorFlow Activation Class

In [209]:
class TensorFlow_Activations:

    def __init__(self, model, x_data, y_data, single_numeric = None, model_name = None):
        self.model = model
        self.x = x_data
        self.y = y_data
        self.model_nm = model_name
        self.signle_num = single_numeric
        self.hid_activ = np.array([])
        self.out_activ = np.array([])
        self.action = self.activations()
        self.action = self.output()


    def inter_activ_hidden(self, data_x):
        '''
        This function takes the x_data as input and return the activations.
        '''
        h_l = tf.keras.Model(self.model.get_layer('Hidden_Layer').input, 
                              self.model.get_layer('Hidden_Layer').output)
        
        return h_l(data_x)
    

    def inter_activ_output(self, data_hidden):
        '''
        This function takes the hidden layer outputs and returns activations for the output layer.
        '''
        o_l = tf.keras.Model(self.model.get_layer('Output_Layer').input, 
                              self.model.get_layer('Output_Layer').output)
        
        return o_l(data_hidden)


    def activations(self):
        '''
        Generates a flattened multi-dimensional array of the activations.
        '''
        
        ### Activations for the hidden layer
        if len(self.x.shape) == 1:
            x_shape = (1,self.x.shape[0])
            x_in = self.x.reshape((1,self.x.shape[0]))
        else:
            x_shape = self.x.shape
            x_in = self.x

        item_h = self.inter_activ_hidden(x_in)

        h_array = np.array(item_h)
        new_shape = (x_shape[0],h_array.shape[1])
        h_array = h_array.flatten().reshape(new_shape)

        self.hid_activ = h_array

        ### Activations for the output layer
        item_o = self.inter_activ_output(item_h)

        o_array = np.array(item_o)
        new_shape = (x_shape[0],o_array.shape[1])
        o_array = o_array.flatten().reshape(new_shape)

        self.out_activ = o_array


    def output(self):

        labels = np.argmax(self.y, axis = 1)

        if self.signle_num == None:

            title1 = f'TensorFlow First 25 Activations for Hidden Layer -- {self.model_nm} Model'
            title2 = f'TensorFlow First 25 Activations for Output Layer -- {self.model_nm} Model'
        
        else:

            title1 = f'TensorFlow First 25 Instances of "{self.signle_num}" Activations for Hidden Layer -- {self.model_nm} Model'
            title2 = f'TensorFlow First 25 Instances of "{self.signle_num}" Activations for Output Layer -- {self.model_nm} Model'

        self.fig1 = activation_heatmap(self.hid_activ[:25], labels, title1).multi_graph()
        self.fig2 = activation_heatmap(self.out_activ[:25], labels, title2).multi_graph()

        return self.hid_activ, self.out_activ, self.fig1, self.fig2
    

    def avg_output(self):

        if self.signle_num != None:
            
            title1 = f'TensorFlow Avg of 25 Instances of "{self.signle_num}" Activations for Hidden Layer -- {self.model_nm} Model'
            title2 = f'TensorFlow Avg of 25 Instances of "{self.signle_num}" Activations for Output Layer -- {self.model_nm} Model'

            self.fig1x = activation_heatmap(self.hid_activ[:25], labels, title1).single_graph_tf()
            self.fig2x = activation_heatmap(self.out_activ[:25], labels, title2).single_graph_tf()

            return self.fig1x, self.fig2x




#### Models

In [192]:
model_name

'mlp_dynamic_03_final_wts'

In [202]:
plt.ioff()
tf_s_run = TensorFlow_Activations(model, xs[:25], ys[:25], single_numeric = numeric_value, model_name = model_name)

/var/folders/jx/zq2lxw9x6xd9gzk5ybm8g2zm0000gn/T/ipykernel_24747/268389681.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(len(self.data_array), sharex = True, figsize = (10,10))


In [233]:
yvals = np.argmax(y_test, axis = 1)
single_index = np.where(yvals == numeric_value)
xs, ys = X_test[single_index], y_test[single_index]

plt.ioff()

TF_ACTIV_DICT = {}

for key, model in tf_model_dict.items():

    ## First 25 Data Points Hidden and Output
    tf_run = TensorFlow_Activations(model, X_test, y_test, model_name = key)
    
    ## Get Hidden Activations First 25
    fig_label = f'TF_{key}_f25_hid.png'
    tf_run.fig1.savefig(fig_label)

    ## Get Output Activations First 25
    fig_label = f'TF_{key}_f25_out.png'
    tf_run.fig2.savefig(fig_label)

    ## Put all activations into Dictionary
    actuals = np.argmax(y_test, axis = 1)
    TF_ACTIV_DICT[key] = {'hidden': tf_run.hid_activ.tolist(), 'output': tf_run.out_activ.tolist(), 'actuals': actuals.tolist()}

    plt.close()

    ## Single Value Activations
    tf_s_run = TensorFlow_Activations(model, xs, ys, single_numeric = numeric_value, model_name = key)

    ## Single Value Activations Hidden Layer
    fig_label = f'TF_{key}_fs_hid.png'
    tf_s_run.fig1.savefig(fig_label)

    ## Single Value Activations Output Layer
    fig_label = f'TF_{key}_fs_out.png'
    tf_s_run.fig2.savefig(fig_label)

    ## Single Value Avg
    f1, f2 = tf_s_run.avg_output()

    ## Single Value Avg Hidden Layer
    fig_label = f'TF_{key}_fs_avg_hid.png'
    f1.savefig(fig_label)

    ## Single Value Avg Output Layer
    fig_label = f'TF_{key}_fs_avg_out.png'
    f2.savefig(fig_label)

    plt.close()


In [234]:
## Save Dictionary

with open("TF_Activations.json","w") as outx:
    json.dump(TF_ACTIV_DICT, outx)

In [242]:
np.array(TF_ACTIV_DICT['base_case_1']['hidden'])[0]

array([0.06870344, 0.3465687 , 0.77779531, 0.29975724, 0.82588786,
       0.06771056, 0.41504976, 0.0192846 , 0.57325661, 0.15579721,
       0.48031461, 0.48373714, 0.15957054, 0.91085893, 0.91453385,
       0.07133066, 0.83271706, 0.16786216, 0.0805342 , 0.37374043,
       0.19244498, 0.31338242, 0.5707202 , 0.22518955, 0.14075969,
       0.42177257, 0.97827131, 0.73420346, 0.11030316, 0.00258984,
       0.75918281, 0.72320396, 0.81358463, 0.11875726, 0.34139639,
       0.97037977, 0.9718473 , 0.80764645, 0.59878546, 0.80945075,
       0.67920411, 0.31916258, 0.76765215, 0.01013992, 0.43757966,
       0.3524504 , 0.93210328, 0.56690371, 0.20304789, 0.90595025,
       0.15391961, 0.62112832, 0.798715  , 0.01399727, 0.2024022 ,
       0.0037299 , 0.99553156, 0.95401496, 0.56033748, 0.14967313,
       0.42206326, 0.8059966 , 0.72539306, 0.34916013, 0.61628711,
       0.15840209, 0.66200179, 0.46102321, 0.06548133, 0.08399698,
       0.83268613, 0.12262796, 0.95963484, 0.94464272, 0.86261

In [ ]:
plt.ion()